In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Sreehari\\Text_Summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Sreehari\\Text_Summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path
    

In [6]:
from textSummarizer.constants import * 
from textSummarizer.logging import logger
from textSummarizer.utils.common import create_directories, read_yaml



In [7]:
class ConfigurationManager:
    def __init__(self,
                config_filepath=CONFIG_FILE_PATH,
                params_filepath=PARAMS_FILE_PATH
    ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
    

In [8]:
import os

In [9]:
import urllib.request as request
import zipfile
from textSummarizer.logging import logger 
from textSummarizer.utils.common import create_directories, get_size



In [10]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config=config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f'{filename} is downloaded with following into \n{headers}')
        else:
            logger.info(f'File already downloaded of size : {get_size(Path(self.config.local_data_file))}')

    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        


In [15]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-09-12 23:02:40,172 : INFO : common : yaml file : config\config.yaml loaded successfully]
[2024-09-12 23:02:40,176 : INFO : common : yaml file : params.yaml loaded successfully]
[2024-09-12 23:02:40,178 : INFO : common : created directory at: artifacts]
[2024-09-12 23:02:40,180 : INFO : common : created directory at: artifacts/data_ingestion]
[2024-09-12 23:02:42,981 : INFO : 2962891892 : artifacts/data_ingestion/data.zip is downloaded with following into 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D70F:11ECE8:AB88F:C83F1:66E325B8
Accept-Ranges: bytes
Date: Thu, 12 Sep 2024 17:32:41 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd11

In [1]:
%pwd

'c:\\Users\\Sreehari\\Text_Summarization\\research'

In [1]:
from textSummarizer.pipeline.stage_01_data_ingestion import DataIngestionTrainingPipeline
from textSummarizer.logging import logger 